In [ ]:
# using ScopeRuntime.Diagnostics;
# using System;
# using System.Collections.Generic;
# using System.Diagnostics;
# using System.IO;
# using System.Linq;
# using System.Text;
# using System.Xml;
# using System.Collections;

# namespace WaqoLib
# {
#     public class Attributes
#     {
#         protected Dictionary<string, string> attrs;

#         internal Attributes()
#         {
#             attrs = new Dictionary<string, string>();
#         }

#         internal void SetAttr(string attrName, string attrValue, bool force = false)
#         {
#             if (!attrs.ContainsKey(attrName))
#                 attrs.Add(attrName, attrValue);
#             else if (force)
#                 attrs[attrName] = attrValue;
#         }

#         internal bool HasAttr(string attrName)
#         {
#             return attrs.ContainsKey(attrName);
#         }

#         internal string GetAttr(string attrName)
#         {
#             if (attrs.ContainsKey(attrName))
#             {
#                 return attrs[attrName];
#             }
#             else
#             {
#                 throw new KeyNotFoundException("ScopeInternalInfo ParseError: critical attribute \"" + attrName + "\" is not found.");
#             }
#         }

#         internal string SafeAttr(string attrName)
#         {
#             if (attrs.ContainsKey(attrName))
#             {
#                 return attrs[attrName];
#             }
#             else
#             {
#                 //DebugStream.WriteLine("ScopeInternalInfo ParseWarning: option attribute \"" + attrName + "\" is not found.");
#                 return null;
#             }
#         }
#     }

#     public class Op
#     {
#         protected readonly Attributes Attrs;
#         protected List<Op> ChildrenList = new List<Op>(1);
#         protected int Exprno = -1;
#         protected int Grpno = -1;

#         internal Op(Attributes attrs)
#         {
#             Attrs = attrs;

#             string grpStr = String.Empty;
#             if(Attrs.HasAttr("Grp"))
#             grpStr=Attrs.GetAttr("Grp");

#             if (grpStr == string.Empty || "Unknown".Equals(grpStr)) return;

#             TryParseGroup(grpStr, out Grpno, out Exprno);
#         }

#         public string Id => Attrs.GetAttr("Id");
#         public string Grp => Attrs.GetAttr("Grp");
#         public int GroupNo => Grpno;
#         public int ExprNo => Exprno;

#         public IEnumerable<Op> Children => ChildrenList;

#         public string this[string attr]
#         {
#             get
#             {
#                 return Attrs.SafeAttr(attr);
#             }
#         }

#         internal void AddChild(Op opChild)
#         {
#             ChildrenList.Add(opChild);
#         }

#         public void SetAttribute(string name, string value, bool force=false)
#         {
#             Attrs.SetAttr(name, value, force);
#         }

#         public override int GetHashCode()
#         {
#             return Attrs.GetAttr("Grp").GetHashCode();
#         }

#         public static bool TryParseGroup(string s, out int grp, out int expr)
#         {
#             grp = -1;
#             expr = -1;

#             if (string.IsNullOrEmpty(s) || "Unknown".Equals(s, StringComparison.OrdinalIgnoreCase))
#                 return false;

#             var tmp = s.Split('.');
#             if (tmp.Length != 2)
#                 return false;

#             return int.TryParse(tmp[0], out grp) && int.TryParse(tmp[1], out expr);
#         }
#     }

#     public class LogOp : Op
#     {
#         internal LogOp(Attributes attrs)
#             : base(attrs)
#         {
#         }

#         public new IEnumerable<LogOp> Children => ChildrenList.Cast<LogOp>();

#         public string Md5 { get; set; }

#         public new Attributes Attrs => base.Attrs;

#         internal void AddChild(LogOp opChild)
#         {
#             ChildrenList.Add(opChild);
#         }

#         public override int GetHashCode()
#         {
#             return GroupNo.GetHashCode();
#         }
#     }

#     public class OpTag : IEquatable<OpTag>
#     {
#         public ulong Ht { get; set; }
#         public byte OpId { get; internal set; }
#         public sbyte Lg { get; internal set; }
#         public uint NumDescendants { get; internal set; }
#         public ulong Htchildren { get; internal set; }

#         public bool Equals(OpTag other)
#         {
#             if (other == null) return false;
#             return Ht == other.Ht && OpId == other.OpId && Lg == other.Lg && NumDescendants == other.NumDescendants &&
#                    Htchildren == other.Htchildren;
#         }

#         public override int GetHashCode()
#         {
#             // ReSharper disable once NonReadonlyMemberInGetHashCode
#             return Ht.GetHashCode();
#         }

#         public override bool Equals(object obj)
#         {
#             return Equals(obj as OpTag);
#         }

#         public static bool TryParse(string s, out OpTag tag)
#         {
#             tag = null;

#             if (s == null)
#             {
#                 return false;
#             }

#             var htComponents = s.Split(':');
#             if (htComponents.Length != 5)
#             {
#                 return false;
#             }

#             ulong ht;
#             byte opid;
#             sbyte lg = -1;
#             uint nDescendants;
#             ulong htChildren;

#             if (!ulong.TryParse(htComponents[0], out ht) || !byte.TryParse(htComponents[1], out opid) ||
#                 !string.IsNullOrEmpty(htComponents[2]) && sbyte.TryParse(htComponents[2], out lg) ||
#                 !uint.TryParse(htComponents[3], out nDescendants) || !ulong.TryParse(htComponents[4], out htChildren))
#                 return false;

#             if (string.IsNullOrEmpty(htComponents[2]))
#                 lg = -1;

#             tag = new OpTag { Ht = ht, OpId = opid, Lg = lg, NumDescendants = nDescendants, Htchildren = htChildren };

#             return true;
#         }

#         public override string ToString()
#         {
#             var buf = new StringBuilder(128);
#             buf.Append(Ht).Append(':').Append(OpId).Append(':');
#             if (Lg >= 0) buf.Append(Lg);
#             buf.Append(':').Append(NumDescendants).Append(':').Append(Htchildren);
#             return buf.ToString();
#         }
#     }

#     public class HtnTag
#     {
#         public ulong HtRoot { get; internal set; }
#         public ulong HtOpCount { get; internal set; }
#         public ulong HtNormInput { get; internal set; }

#         public static bool TryParse(string s, out HtnTag tag)
#         {
#             tag = null;

#             if (s == null)
#             {
#                 return false;
#             }

#             var htComponents = s.Split(':');
#             if (htComponents.Length != 3)
#             {
#                 return false;
#             }

#             ulong htRoot, htOpCount, htNormInput;
#             if (!ulong.TryParse(htComponents[0], out htRoot) || !ulong.TryParse(htComponents[1], out htOpCount) || !ulong.TryParse(htComponents[2], out htNormInput))
#             {
#                 return false;
#             }

#             tag = new HtnTag { HtRoot = htRoot, HtOpCount = htOpCount, HtNormInput = htNormInput };
#             return true;
#         }
#     }

#     public class PhyOp : Op
#     {
#         internal PhyOp(Attributes attrs)
#             : base(attrs)
#         {
#             HtLogOpComb = new HashSet<string>();
#             InputCardinality = 0;
#             Hts = 0;
#             Htt = 0;
# 	        Hti = 0;
#             Tag = null;
#             Cardinality = double.NaN;

#             ulong hts;
#             ulong htt;
#             OpTag tag;
#             HtnTag htn;
#             double card;
# 	        ulong v;


#             if (ulong.TryParse(attrs.SafeAttr("HTS"), out hts))
#             {
#                 Hts = hts;
#             }

#             if (ulong.TryParse(attrs.SafeAttr("HTT"), out htt))
#             {
#                 Htt = htt;
#             }

#             if (OpTag.TryParse(attrs.SafeAttr("HT"), out tag))
#             {
#                 Tag = tag;
#             }

#             if (HtnTag.TryParse(attrs.SafeAttr("HTN"), out htn))
#             {
#                 Htn = htn;
#             }

#             if (ulong.TryParse(attrs.SafeAttr("HTI"), out v))
#                 Hti = v;

#             if (double.TryParse(attrs.SafeAttr("Card"), out card))
#                 Cardinality = card;
#         }


#         public Dictionary<string, string> PhyOp_To_LogOp = new Dictionary<string, string>()
#         {
#             {"PhyOp_Extract","LogOp_Extract"},
#             {"PhyOp_ExtractIntermediate","Logop_ExtractRemote"},
#             {"PhyOp_ExtractRemote","Logop_ExtractRemote"},
#             {"PhyOp_Output","LogOp_Output"},
#             {"PhyOp_Range","LogOp_Get"},
#             {"PhyOp_Write","LogOp_Write"},
#             {"PhyOp_Filter","LogOp_Select"},
#             {"PhyOp_ComputeScalar","LogOp_RestrRemap"},
#             {"PhyOp_RestrRemap","LogOp_RestrRemap"},
#             {"PhyOp_HashGbAgg","LogOp_GbAgg"},
#             {"PhyOp_StreamGbAgg","LogOp_GbAgg"},
#             {"PhyOp_StreamRollup","LogOp_Rollup"},
#             {"PhyOp_Reduce","LogOp_Reduce"},
#             {"PhyOp_LoopsJoin","LogOp_Join"},
#             {"PhyOp_HashJoin","LogOp_Join"},
#             {"PhyOp_Combine","LogOp_Union"},
#             {"PhyOp_MergeJoin","LogOp_Join"},
#             {"PhyOp_IdxLookup","LogOp_GetIdx"},
#             {"PhyOp_GbApply","LogOp_GbApply"},
#             {"PhyOp_Window","LogOp_Process"},
#             {"PhyOp_Top","LogOp_Top"},
#             {"PhyOp_ComputeSequence","LogOp_SequenceProject"},
#             {"PhyOp_Process","LogOp_Process"},
#             {"PhyOp_Sampler","LogOp_Sampler"},
#             {"PhyOp_MultiProcess","LogOp_MultiProcess"},
#             {"PhyOp_DeclareContext","LogOp_DeclareContext"},
#             {"PhyOp_Spool","LogOp_Spool"},
#             {"PhyOp_Sequence","LogOp_SequenceProject"},
#             {"PhyOp_UnionAll","LogOp_UnionAll"},
#             {"PhyOp_VirtualDataset","LogOp_UnionAll"},
#             {"PhyOp_ConstTableScan","LogOp_ConstTableGet"},
#             {"PhyOp_Update","LogOp_Update"},
#             {"PhyOp_Sort","LogOp_OrderBy"}
#         };

#         public Dictionary<String, ulong> LogOps_Count;
#         public HashSet<String> HtLogOpComb;

#         public string Ht => Attrs.GetAttr("HT");
#         public ulong Hts { get; }
#         public ulong Htt { get; }
#         public ulong Hti { get; }
#         public bool AboveScan { get; set; }

#         public OpTag Tag { get; }
#         public HtnTag Htn { get; }
#         public double Cardinality { get; }
#         // Added by Chenggang. Keep track of the input cardinality of the subtree
#         public double InputCardinality { get; set; }

#         public RuntimeStats RuntimeStat { get; set; }

#         public new IEnumerable<PhyOp> Children => ChildrenList.Cast<PhyOp>();

#         internal void AddChild(PhyOp opChild)
#         {
#             ChildrenList.Add(opChild);
#         }

#         public void UpdateLogOps_Count(PhyOp childPhyOp)
#         {
#             ulong currentValue = 0;
#             if (childPhyOp.LogOps_Count!=null)
#             {
                
#                 foreach (KeyValuePair<string, ulong> entry in childPhyOp.LogOps_Count)
#                 {
#                     if (HtLogOpComb.Contains(childPhyOp.Hts.ToString() + ":" + entry.Key))
#                         continue;
#                     HtLogOpComb.Add(childPhyOp.Hts.ToString() + ":" + entry.Key);
#                     currentValue = 0;
#                     if (LogOps_Count == null)
#                         LogOps_Count = new Dictionary<string, ulong>();
#                     LogOps_Count.TryGetValue(entry.Key, out currentValue);
#                     if(LogOps_Count.ContainsKey(entry.Key))
#                      LogOps_Count[entry.Key] = (entry.Value + currentValue);
#                     else
#                       LogOps_Count.Add(entry.Key, entry.Value);

#                 }
#             }
      
#             string childlogop = null;
#             PhyOp_To_LogOp.TryGetValue(childPhyOp["Id"], out childlogop);
           

#             currentValue = 0;
#             if (childlogop != null && !HtLogOpComb.Contains(childPhyOp.Hts.ToString() + ":" + childlogop))
#             {
#                 HtLogOpComb.Add(childPhyOp.Hts.ToString() + ":" + childlogop);

#                 if (LogOps_Count == null)
#                     LogOps_Count = new Dictionary<string, ulong>();
               
#                 LogOps_Count.TryGetValue(childlogop, out currentValue);
#                 if (LogOps_Count.ContainsKey(childlogop))
#                     LogOps_Count[childlogop] = (currentValue + 1);
#                 else
#                     LogOps_Count.Add(childlogop, 1);
#             }
       
               
#         }

#     }

#     public class AncOp : Op
#     {
#         internal AncOp(Attributes attrs)
#             : base(attrs)
#         {
#         }

#         public new Attributes Attrs => base.Attrs;
#     }

#     public class ScaOp : Op
#     {
#         internal ScaOp(Attributes attrs)
#             : base(attrs)
#         {
#         }

#         public new Attributes Attrs => base.Attrs;
#     }

#     public abstract class OpTreeParser
#     {
#         private readonly Dictionary<int, LogOp> _inputLogOpCache = new Dictionary<int, LogOp>();
#         private readonly Dictionary<int, LogOp> _logOpCache = new Dictionary<int, LogOp>();
#         private readonly Dictionary<string, PhyOp> _phyOpCache = new Dictionary<string, PhyOp>();

#         protected Stack<Op> Stack = new Stack<Op>(1024);
#         protected string CurrentParamKey;

#         protected virtual bool ParseInputTree => true;
#         protected virtual bool ParseLogicalTree => true;
#         protected virtual bool ParseOutputTree => true;

#         public LogOp InputTreeRoot { get; private set; }
#         public LogOp LogicalTreeRoot { get; private set; }
#         public PhyOp OutputTreeRoot { get; private set; }

#         // represents a bit vector for the rules triggered in this plan
#         public byte[] RulesUsed { get; private set; }

#         public LogOp GetLogOp(int grp, bool input)
#         {
#             LogOp op;
#             return (input ? _inputLogOpCache : _logOpCache).TryGetValue(grp, out op) ? op : null;
#         }

#         public LogOp GetLogOp(int grp)
#         {
#             LogOp op;
#             return _inputLogOpCache.TryGetValue(grp, out op) || _logOpCache.TryGetValue(grp, out op) ? op : null;
#         }

#         public PhyOp GetPhyOp(string grp)
#         {
#             PhyOp op;
#             return _phyOpCache.TryGetValue(grp, out op) ? op : null;
#         }

#         protected void ReadAttributes(XmlReader reader, Attributes attrs, string attrOption = "")
#         {
#             if (!reader.HasAttributes) return;
#             while (reader.MoveToNextAttribute())
#                 attrs.SetAttr(reader.Name + attrOption, reader.Value);

#             // Move the reader back to the element node.
#             reader.MoveToElement();
#         }

#         public bool Parse(Stream stream)
#         {
#             using (var reader = SafeXml.CreateXmlReaderFromStream(stream))
#             {
#                 return Parse(reader);
#             }
#         }

#         public bool Parse(TextReader textReader)
#         {
#             using (var reader = SafeXml.CreateXmlReaderFromReader(textReader))
#             {
#                 return Parse(reader);
#             }
#         }

#         private void VisitLogOpTree(XmlReader reader, bool input)
#         {
#             var inTree = true;
#             var endTag = input ? "InputTree" : "LogicalTree";
#             var opCache = input ? _inputLogOpCache : _logOpCache;
#             var isRoot = true;
#             while (inTree && reader.Read())
#                 switch (reader.NodeType)
#                 {
#                     case XmlNodeType.Element:
#                         if ("LogOp".Equals(reader.Name))
#                         {
#                             var attrs = new Attributes();
#                             ReadAttributes(reader, attrs);
#                             reader.ReadToDescendant("TableGroupProperties");
#                             ReadAttributes(reader, attrs);

#                             var grp = attrs.GetAttr("Grp");
#                             int grpNo;
#                             int exprNo;
#                             LogOp op;

#                             if (Op.TryParseGroup(grp, out grpNo, out exprNo))
#                             {
#                                 if (!opCache.TryGetValue(grpNo, out op))
#                                 {
#                                     op = new LogOp(attrs);
#                                     opCache.Add(grpNo, op);

#                                     if (isRoot)
#                                     {
#                                         if (input)
#                                         {
#                                             InputTreeRoot = op;
#                                         }
#                                         else
#                                         {
#                                             LogicalTreeRoot = op;
#                                         }

#                                         isRoot = false;
#                                     }

#                                     OnLogOp(attrs, input);
#                                 }
#                             }
#                             else
#                             {
#                                 op = new LogOp(attrs);

#                                 // populating the opCache since it is used in the end to test whether parsing succeeded
#                                 opCache.Add(opCache.Count() == 0 ? -1 : opCache.Keys.Max() + 1, op);

#                                 OnLogOp(attrs, input);
#                             }

#                             if (Stack.Count > 0)
#                             {
#                                 Stack.Peek().AddChild(op);
#                             }

#                             Stack.Push(op);
#                         }
#                         else if ("AncOp".Equals(reader.Name))
#                         {
#                             var attrs = new Attributes();
#                             ReadAttributes(reader, attrs);
#                             var op = new AncOp(attrs);
#                             Stack.Push(op);
#                         }
#                         else if ("ScaOp".Equals(reader.Name))
#                         {
#                             var attrs = new Attributes();
#                             ReadAttributes(reader, attrs);
#                             var op = new ScaOp(attrs);
#                             Stack.Push(op);
#                         }
#                         break;

#                     case XmlNodeType.EndElement:
#                         if ("LogOp".Equals(reader.Name))
#                         {
#                             // now we can visit this node.
#                             var op = Stack.Pop() as LogOp;
#                             if (op != null) VisitLogOp(op, op.Children, input);

#                             //var newStack = new Stack<Op>(stack);
#                         }

#                         if ("AncOp".Equals(reader.Name))
#                         {
#                             // now we can visit this node.
#                             var op = Stack.Pop() as AncOp;
#                             VisitAncOp(op, null);
#                         }

#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             // now we can visit this node.
#                             var op = Stack.Pop() as ScaOp;
#                             VisitScaOp(op, null);
#                         }
#                         else if (endTag.Equals(reader.Name))
#                         {
#                             inTree = false;
#                         }
#                         break;
#                 }
#         }

#         private void VisitOpInfo(XmlReader reader, Attributes attrs)
#         {
#             switch (reader.Name)
#             {
#                 case "TableGroupProperties":
#                     ReadAttributes(reader, attrs);
#                     break;

#                 case "Costs":
#                     ReadAttributes(reader, attrs);
#                     break;

#                 case "DrvdTablePlanProperties":
#                     ReadAttributes(reader, attrs);
#                     break;

#                 case "ReqdTablePlanProperties":
#                     ReadAttributes(reader, attrs, "REQ:");
#                     break;
#             }
#         }

#         private void VisitPhyOpTree(XmlReader reader)
#         {
#             var inOutputTree = true;
#             var isRoot = true;
#             while (inOutputTree && reader.Read())
#             {
#                 switch (reader.NodeType)
#                 {
#                     case XmlNodeType.Element:
#                         if ("PhyOp".Equals(reader.Name))
#                         {
#                             var attrs = new Attributes();
#                             ReadAttributes(reader, attrs);

#                             if (reader.ReadToDescendant("OpInfo"))
#                             {
#                                 while (reader.Read() && !("OpInfo".Equals(reader.Name)))
#                                 {
#                                     VisitOpInfo(reader, attrs);
#                                 }
#                             }

#                             var grp = attrs.GetAttr("Grp");
#                             PhyOp op;
#                             if (!_phyOpCache.TryGetValue(grp, out op))
#                             {
#                                 op = new PhyOp(attrs);
#                                 _phyOpCache.Add(grp, op);

#                                 if (isRoot)
#                                 {
#                                     OutputTreeRoot = op;
#                                     isRoot = false;
#                                 }
                                
#                                 // added another constraint here for
#                                 // PhyOp_Extract
#                                 if (attrs.HasAttr("Table") 
#                                     && !attrs.GetAttr("Id").Equals("PhyOp_Output") 
#                                     && !attrs.GetAttr("Id").Equals("PhyOp_Write")
#                                     && !attrs.GetAttr("Id").Equals("PhyOp_Update")
#                                     && !attrs.GetAttr("Id").Equals("PhyOp_Extract"))
#                                 {
#                                     OnPhyOp(attrs, op);
#                                 }
#                             }
#                             else
#                             {
#                                 if (isRoot)
#                                 {
#                                     OutputTreeRoot = op;
#                                     isRoot = false;
#                                 }
#                                 if (attrs.GetAttr("Id").Equals("PhyOp_Spool"))
#                                 {
#                                     OnPhyOp(attrs, op);
#                                 }
#                             }

#                             if (Stack.Count > 0)
#                             {
#                                 Stack.Peek().AddChild(op);
#                             }

#                             Stack.Push(op);
#                         }
#                         else if ("ScaOp".Equals(reader.Name))
#                         {
#                             var attrs = new Attributes();
#                             ReadAttributes(reader, attrs);
#                             OnScaOp(attrs);
#                             Expression expression = null;

#                             try
#                             {
#                                 if (Stack.Count > 0 && Stack.Peek().Id.Equals("PhyOp_Filter"))
#                                 {
#                                     expression = ReadExpression(reader);
#                                     Expression nrmdExpr = ExpressionUtil.NormalizeToCnfUtil(expression);
#                                     Expression fltNrmdExpr = ExpressionUtil.FlattenAndSortCNFExpression(nrmdExpr, 0);
#                                     Expression fltNrmdExprWoValues = fltNrmdExpr.DeepClone(false, 0);
#                                     List<string> placeHolderMappedValues = new List<string>();
#                                     Expression fltNrmdExprwithPlaceHolders = fltNrmdExpr.ReplaceValuesWithPlaceHolder(placeHolderMappedValues, 0);

#                                     List<string> expressionList = ExpressionUtil.GeneratePredicateList(fltNrmdExpr, 0);
#                                     string exprString = Newtonsoft.Json.JsonConvert.SerializeObject(expression);
#                                     string fltNrmdExpressionString = Newtonsoft.Json.JsonConvert.SerializeObject(fltNrmdExpr);
#                                     string fltNrmdExprWoValuesString = Newtonsoft.Json.JsonConvert.SerializeObject(fltNrmdExprWoValues);
#                                     string fltNrmdExprwithPlaceHoldersString = Newtonsoft.Json.JsonConvert.SerializeObject(fltNrmdExprwithPlaceHolders);
#                                     int exprHashCode = fltNrmdExpr.GetHashCode();
#                                     int exprwoValuesHashCode = fltNrmdExprWoValues.GetHashCode();
#                                     string colonSeparatedPredicateList = string.Join(";", expressionList);
#                                     string placeHolderMappedValuesStr = string.Join("$;$", placeHolderMappedValues);

#                                     using (var e = Stack.GetEnumerator())
#                                     {
#                                         while (e.MoveNext())
#                                         {
#                                             if (e.Current == null) continue;
#                                             if (e.Current["FilterExpr"] == null)
#                                             {
#                                                 e.Current.SetAttribute("FilterExpr", exprString);
#                                             }
#                                             if (e.Current["FilterExprNorm"] == null)
#                                             {
#                                                 e.Current.SetAttribute("FilterExprNorm", fltNrmdExpressionString);
#                                             }
#                                             if (e.Current["FilterExprNormWoValues"] == null)
#                                             {
#                                                 e.Current.SetAttribute("FilterExprNormWoValues", fltNrmdExprWoValuesString);
#                                             }
#                                             if (e.Current["FilterExprNormHash"] == null)
#                                             {
#                                                 e.Current.SetAttribute("FilterExprNormHash", exprHashCode.ToString());
#                                             }
#                                             if (e.Current["FilterExprNormWoValuesHash"] == null)
#                                             {
#                                                 e.Current.SetAttribute("FilterExprNormWoValuesHash", exprwoValuesHashCode.ToString());
#                                             }
#                                             if (e.Current["Parameters"] == null)
#                                             {
#                                                 e.Current.SetAttribute("Parameters", colonSeparatedPredicateList);
#                                             }
#                                             if (e.Current["FltNrmdExprwithPlaceHolders"] == null)
#                                             {
#                                                 e.Current.SetAttribute("FltNrmdExprwithPlaceHolders", fltNrmdExprwithPlaceHoldersString);
#                                             }
#                                             if (e.Current["PlaceHolderMappedValues"] == null)
#                                             {
#                                                 e.Current.SetAttribute("PlaceHolderMappedValues", placeHolderMappedValuesStr);
#                                             }
#                                         }
#                                     }
#                                 }
#                             }
#                             catch (Exception ex)
#                             {
#                                 DebugStream.WriteLine("Error parsing filter expression", string.Format("{0}", ex));
#                             }

#                             if (expression == null)
#                             {
#                                 attrs.SetAttr("FilterExpr", "");
#                                 attrs.SetAttr("FilterExprNorm", "");
#                                 attrs.SetAttr("FilterExprNormWoValues", "");
#                                 attrs.SetAttr("FilterExprNormHash", "0");
#                                 attrs.SetAttr("FilterExprNormWoValuesHash", "0");
#                                 attrs.SetAttr("FltNrmdExprwithPlaceHolders", "");
#                                 attrs.SetAttr("PlaceHolderMappedValues", "");
#                                 ReadAttributes(reader, attrs);
#                                 OnScaOp(attrs);
#                             }
#                         }
#                         break;

#                     case XmlNodeType.EndElement:
#                         if ("PhyOp".Equals(reader.Name))
#                         {
#                             // now we can visit this node.
#                             var op = Stack.Pop() as PhyOp;
#                             PhyOp parent;
#                             if (Stack.Count > 0)
#                             {
#                                 parent = Stack.Peek() as PhyOp;
#                             }
#                             else
#                             {
#                                 parent = null;
#                             }

#                             if (op != null)
#                             {
#                                 VisitPhyOp(op, op.Children, parent);
#                             }
#                         }
#                         else if ("OutputTree".Equals(reader.Name))
#                         {
#                             inOutputTree = false;
#                         }
#                         break;
#                 }
#             }
#         }

#         public Expression ReadLogicalExpression(XmlReader reader)
#         {
#             string op = reader.GetAttribute("LogicalOp");
#             Expression expression = new Expression
#             {
#                 isComparison = false
#             };
#             if (op.Contains("And"))
#                 expression.expOperator = "And";
#             else if (op.Contains("Or"))
#                 expression.expOperator = "Or";

#             while (reader.Read())
#             {
#                 switch (reader.NodeType)
#                 {
#                     case XmlNodeType.Element:
#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             Expression childExpression = ReadExpression(reader);
#                             expression.children.Add(childExpression);

#                         }
#                         break;

#                     case XmlNodeType.EndElement:
#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             reader.MoveToElement();
#                             return expression;
#                         }
#                         break;
#                 }

#             }

#             // should never reach this point
#             return null;
#         }

#         public Expression ReadComparisonExpression(XmlReader reader)
#         {
#             Expression expression = new Expression
#             {
#                 isComparison = true
#             };
#             string op = reader.GetAttribute("CompOp");
#             switch (op)
#             {
#                 case "x_cmpGe": expression.expOperator = ">="; break;
#                 case "x_cmpGt": expression.expOperator = ">"; break;
#                 case "x_cmpNe": expression.expOperator = "!="; break;
#                 case "x_cmpLe": expression.expOperator = "<="; break;
#                 case "x_cmpLt": expression.expOperator = "<"; break;
#                 case "x_cmpEq": expression.expOperator = "="; break;
#             }
#             expression.attribute = new Attribute("", "");
#             int elementsOpen = 1;
#             while (reader.Read())
#             {
#                 switch (reader.NodeType)
#                 {
#                     case XmlNodeType.Element:

#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             elementsOpen += 1;
#                             string CompId = reader.GetAttribute("Id");
#                             if (CompId.Contains("Identifier"))
#                             {
#                                 expression.attribute.name = reader.GetAttribute("Column");
#                             }
#                             if (CompId.Contains("Const"))
#                             {
#                                 expression.values.Add(reader.GetAttribute("Value"));
#                                 string T1 = reader.GetAttribute("TI");
#                                 string type;
#                                 if (T1.Contains(","))
#                                 {
#                                     type = T1.Split(',')[0];
#                                 }
#                                 else
#                                 {
#                                     type = T1;
#                                 }
#                                 expression.attribute.dataType = type;
#                             }
#                         }
#                         break;

#                     case XmlNodeType.EndElement:

#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             elementsOpen -= 1;
#                             if (elementsOpen == 0)
#                             {
#                                 reader.MoveToElement();
#                                 return expression;
#                             }
#                         }

#                         break;

#                 }

#             }

#             // should never reach this point
#             return null;

#         }


#         public Expression ReadUdfExpression(XmlReader reader)
#         {
#             Expression expression = new Expression
#             {
#                 isComparison = true,
#                 expOperator = reader.GetAttribute("Udf")
#             };
#             int elementsOpen = 1;
#             while (reader.Read())
#             {
#                 switch (reader.NodeType)
#                 {
#                     case XmlNodeType.Element:
#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             elementsOpen += 1;
#                         }
#                         break;
#                     case XmlNodeType.EndElement:
#                         if ("ScaOp".Equals(reader.Name))
#                         {
#                             elementsOpen -= 1;
#                             if (elementsOpen == 0)
#                             {
#                                 reader.MoveToElement();
#                                 return expression;
#                             }
#                         }
#                         break;
#                 }
#             }
#             // should never reach this point
#             return null;
#         }

#         public Expression ReadExpression(XmlReader reader)
#         {
#             if (null == reader.GetAttribute("Id"))
#                 return null;
#             string id = reader.GetAttribute("Id");
#             if (id.Contains("Logical"))
#             {
#                 return ReadLogicalExpression(reader);
#             }
#             else if (id.Contains("Comp"))
#             {
#                 return ReadComparisonExpression(reader);
#             }
#             else if (id.Contains("Udf"))
#             {
#                 return ReadUdfExpression(reader);

#             }
#             // should never reach this point
#             return null;
#         }
        
#         private bool Parse(XmlReader reader)
#         {
#             Reset();

#             Console.WriteLine($"Begin parse internal info file.");
#             var logicalTreePresent = false;

#             // skip until we sees OutputTree
#             while (reader.Read())
#             {
#                 if (reader.NodeType != XmlNodeType.Element) continue;
#                 switch (reader.Name)
#                 {
#                     case "InputTree":
#                         if (ParseInputTree)
#                         {
#                             VisitLogOpTree(reader, true);
#                         }
#                         break;
#                     case "LogicalTree":
#                         logicalTreePresent = true;
#                         if (ParseLogicalTree)
#                         {
#                             VisitLogOpTree(reader, false);
#                         }
#                         break;
#                     case "OutputTree":
#                         if (ParseOutputTree)
#                         {
#                             VisitPhyOpTree(reader);
#                         }
#                         break;
#                     case "Rules":
#                         var attrs = new Attributes();
#                         ReadAttributes(reader, attrs);
#                         String rules = attrs.GetAttr("Used");
#                         // in order to use the BitArray class, we convert to
#                         // boolean array --> BitArray --> ByteArray
#                         bool[] rulesBool = new bool[rules.Length];
#                         for (int i = 0; i < rules.Length; i++) { 
#                             if (rules[i] == '1') {
#                               rulesBool[i] = true;
#                             }
#                             else rulesBool[i] = false;
#                         }

#                         BitArray rulesBits = new BitArray(rulesBool);
#                         byte[] rulesBytes = new byte[(rulesBits.Length - 1) / 8 + 1];
#                         rulesBits.CopyTo(rulesBytes, 0);
#                         RulesUsed = rulesBytes;

#                         BitArray ba2 = new BitArray(rulesBytes);
#                         break;
#                 }
#             }

#             return ParseInputTree == _inputLogOpCache.Any()
#                    && (!logicalTreePresent || ParseLogicalTree == _logOpCache.Any())
#                    && ParseOutputTree == _phyOpCache.Any();
#         }

#         public void Reset()
#         {
#             Stack.Clear();
#             _inputLogOpCache.Clear();
#             _logOpCache.Clear();
#             _phyOpCache.Clear();
#             InputTreeRoot = null;
#             LogicalTreeRoot = null;
#             OutputTreeRoot = null;
#             OnReset();
#         }

#         protected abstract void VisitLogOp(LogOp op, IEnumerable<LogOp> children, bool input);
#         protected abstract void VisitPhyOp(PhyOp op, IEnumerable<PhyOp> children, PhyOp parent);
#         protected abstract void VisitAncOp(AncOp op, IEnumerable<AncOp> children);
#         protected abstract void VisitScaOp(ScaOp op, IEnumerable<ScaOp> children);

#         /// <summary>
#         /// </summary>
#         /// <param name="logOpAttrs">collection of attributes of this logop</param>
#         /// <param name="input">true when traversing input tree</param>
#         protected abstract void OnLogOp(Attributes logOpAttrs, bool input);

#         /// <summary>
#         /// </summary>
#         /// <param name="phyOpAttrs">collection of attributes of this phyop</param>
#         protected abstract void OnPhyOp(Attributes phyOpAttrs, PhyOp op);

#         protected abstract void OnScaOp(Attributes scaOpAttrs);

#         protected virtual void OnReset()
#         {
#         }
#     }

#     /*
#      * Schema
#      * <OutputTree>
#      * ...
#      * <PhyOp Id="PhyOp_xxx" ...>
#      *  <OpInfo>
#      *      <TableGroupProperties Grp="..." ... HT="..." />
#      *      <Costs ... />
#      *  </OpInfo>
#      *  ...
#      * </PhyOp>
#      * ...
#      * </OutputTree>
#      */
#     public abstract class PhyOpTreeParser : OpTreeParser
#     {
#         protected override bool ParseInputTree => false;

#         protected override bool ParseLogicalTree => false;

#         protected override void VisitLogOp(LogOp op, IEnumerable<LogOp> children, bool input)
#         {
#         }

#         protected override void VisitAncOp(AncOp op, IEnumerable<AncOp> children)
#         {
#         }

#         protected override void VisitScaOp(ScaOp op, IEnumerable<ScaOp> children)
#         {
#         }

#         protected override void OnLogOp(Attributes logOpAttrs, bool input)
#         {
#         }
#     }

#     public class DefaultPhyOpTreeParser : PhyOpTreeParser
#     {
#         protected override void VisitPhyOp(PhyOp op, IEnumerable<PhyOp> children, PhyOp parent)
#         {
#         }

#         protected override void OnPhyOp(Attributes phyOpAttrs, PhyOp op)
#         {
#         }

#         protected override void OnScaOp(Attributes scaOpAttrs)
#         {
#         }

#         protected override void VisitAncOp(AncOp op, IEnumerable<AncOp> children)
#         {
#         }

#         protected override void VisitScaOp(ScaOp op, IEnumerable<ScaOp> children)
#         {
#         }
#     }

#     public class PhyOpTreeParserWithHtsCache : PhyOpTreeParser
#     {
#         public Dictionary<ulong, PhyOp> HtStoPhyOp { get; } = new Dictionary<ulong, PhyOp>();

#         protected override void VisitPhyOp(PhyOp op, IEnumerable<PhyOp> children, PhyOp parent)
#         {
#             PhyOp opCached;
#             if (HtStoPhyOp.TryGetValue(op.Hts, out opCached))
#             {
#                 Debug.Assert(opCached == op);
#             }
#             else
#             {
#                 HtStoPhyOp.Add(op.Hts, op);
#             }
#         }

#         protected override void VisitAncOp(AncOp op, IEnumerable<AncOp> children)
#         {
#         }

#         protected override void VisitScaOp(ScaOp op, IEnumerable<ScaOp> children)
#         {
#         }

#         protected override void OnPhyOp(Attributes phyOpAttrs, PhyOp op)
#         {
#         }

#         protected override void OnScaOp(Attributes scaOpAttrs)
#         {
#         }

#         protected override void OnReset()
#         {
#             HtStoPhyOp.Clear();
#         }
#     }
# }
